#**Import Dependencies**

In [ ]:
!pip install finta
!pip install oandapyV20

     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for oandapyV20: filename=oandapyV20-0.7.0-cp37-none-any.whl size=70015 sha256=268fd87b2910f4afe00b956c941d106ef6c25a6614ad2f48f14b309d518f5586
  Stored in directory: /root/.cache/pip/wheels/a2/a0/2a/d96bbeddbdc532de319532d28787daddcf64459626eb1c05be
Successfully built oandapyV20


In [ ]:
from finta import TA

In [ ]:
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.forexlabs as labs
import oandapyV20

accountID = "your account id"
access_token = "your access token"

client = oandapyV20.API(access_token=access_token)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
import pickle

#**Data Prepering**

In [ ]:
def get_data(symbol, timeframe, count):
    params = {"count": count, "granularity": timeframe}
    r = instruments.InstrumentsCandles(instrument=symbol, params=params)
    client.request(r)
    candles = r.response['candles']

    df = pd.json_normalize(candles)
    df['High'] = df['mid.h'].astype('float64').dropna()
    df['Low'] = df['mid.l'].astype('float64').dropna()
    df['Open'] = df['mid.o'].astype('float64').dropna()
    df['Close'] = df['mid.c'].astype('float64').dropna()
    df['Volume'] = df['mid.c'].astype('float64').dropna()
    df = df[['Open' , 'Close' , 'High' , 'Low' , 'Volume']]
    return df

In [ ]:
df = get_data('EUR_USD' , 'D' , 5000)

df['SMA_FAST'] = TA.SMA(df,14)
df['SMA_SLOW'] = TA.SMA(df,21)
previous_10 = df['SMA_FAST'].shift(1)
previous_20 = df['SMA_SLOW'].shift(1)
df['MA_CROSSOVER'] = crossing = (((df['SMA_FAST'] <= df['SMA_SLOW']) & (previous_10 >= previous_20))
            | ((df['SMA_FAST'] >= df['SMA_SLOW']) & (previous_10 <= previous_20)))
df['RSI'] = TA.RSI(df)
df['ADX'] = TA.ADX(df)
df['CCI'] = TA.CCI(df)
df['STOCH'] = TA.STOCH(df)
df['MACD'] = TA.MACD(df)['MACD']
df['ATR'] = TA.ATR(df)
df['CHANGE'] = df['Close'].diff()
df['OBV'] = TA.OBV(df)
df['Prediction'] = df['Close'].transform(lambda x : np.sign(x.diff()))
df.loc[df['Prediction'] == 0.0] = 1.0
df['Prediction'] = df['Prediction'].shift(-1)

df.fillna(0,inplace=True)

In [ ]:
df.head(5)

,Open,Close,High,Low,Volume,SMA_FAST,SMA_SLOW,MA_CROSSOVER,RSI,ADX,CCI,STOCH,MACD,ATR,CHANGE,OBV,Prediction
0,1.24215,1.24215,1.24215,1.24215,1.24215,0.0,0.0,False,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.00000,-1.0
1,1.23880,1.23880,1.23880,1.23880,1.23880,0.0,0.0,False,0.000000,0.0,-66.666667,0.0,-0.000075,0.0,-0.00335,-1.23880,-1.0
2,1.23278,1.23278,1.23278,1.23278,1.23278,0.0,0.0,False,0.000000,0.0,-100.000000,0.0,-0.000282,0.0,-0.00602,-2.47158,1.0
3,1.23470,1.23470,1.23470,1.23470,1.23470,0.0,0.0,False,18.464165,0.0,-47.661470,0.0,-0.000300,0.0,0.00192,-1.23688,1.0
4,1.23900,1.23900,1.23900,1.23900,1.23900,0.0,0.0,False,43.586686,0.0,33.680370,0.0,-0.000133,0.0,0.00430,0.00212,1.0


#**Machine Learning MOdel**

In [ ]:
X_Cols = df[['CHANGE','RSI','STOCH','MACD','ADX','CCI','OBV' , 'MA_CROSSOVER']]
Y_Cols = df['Prediction']

# Split X and y into X_
X_train, X_test, y_train, y_test = train_test_split(X_Cols, Y_Cols, random_state = 0 , test_size=0.2)

# Create a Random Forest Classifier
rand_frst_clf = RandomForestClassifier(n_estimators = 100, oob_score = True, criterion = "gini", random_state = 0)

# Fit the data to the model
rand_frst_clf.fit(X_train, y_train)

# Make predictions
y_pred = rand_frst_clf.predict(X_test)

In [ ]:
print('Correct Prediction (%): ', accuracy_score(y_test, rand_frst_clf.predict(X_test), normalize = True) * 100.0)

Correct Prediction (%):  52.300000000000004


In [ ]:
filename = 'fx_rf_model.sav'
pickle.dump(rand_frst_clf, open(filename, 'wb'))